In [1]:
import sys
!{sys.executable} -m pip install langchain
!{sys.executable} -m pip install openai
!{sys.executable} -m pip install pandas

In [2]:
from langchain.prompts import (
    PromptTemplate,
    FewShotPromptTemplate,
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI

#import openai
#from langchain import *
#from pydantic import *
import os 
import pandas as pd 

with open("secret.txt") as f:
    key=f.read()
os.environ['OPENAI_API_KEY']=key[:-1]


In [3]:


example_formatter = """sentence: {sentence}
Tag: {tag}"""

example_prompt = PromptTemplate(
    input_variables=["sentence", "tag"],
    template=example_formatter
)

suffix = "sentence: {sentence}\nTag:"

body_prefix = """Classify the following as one of:

1. body: if the description is about body parts
2. not body: if the description does not mention body parts """

shape_prefix = """Classify the following as one of:

1. shape: if the description references geometric shapes
2. not shape: if the description does not reference geometric shapes"""

abstract_prefix = """Classify the following as one of:

1. abstract: if the description is in terms of a type of person or creature
2. not abstract: if the description is concrete and not metaphorical """


def make_chat_template(instruction):
    system_message = SystemMessage(content=instruction)
    messages = []
    #for example in examples:
    #   messages.append(HumanMessage(content=example["sentence"]))
    #   messages.append(AIMessage(content=example["tag"]))
    messages.append(HumanMessagePromptTemplate.from_template("{test_sentence}"))

    return ChatPromptTemplate.from_messages([system_message, *messages])

body_chat = make_chat_template(body_prefix)
shape_chat = make_chat_template(shape_prefix)
abstract_chat = make_chat_template(abstract_prefix)

    

In [13]:



def tag_data_with_langchain(df, model, prompt): 
  
  res = []
  #for index in range(100):
  for index, _ in df.iterrows():
    print(index)
    #print(model)
    #print(prompt)
    chain=LLMChain(llm=model,prompt=prompt)
    output=chain.run(df["gpt_out"][index])
    print(df["gpt_out"][index])
    #string_output=output.content
    print(output)
    res.append(output)
  return res





In [15]:
model_name = "gpt-3.5-turbo"
#model_name="gpt-4"
temperature = 0
llm = ChatOpenAI(model_name=model_name, temperature=0)

chain=LLMChain
input =pd.read_csv("data/test/sample_to_tag.csv")

test_input=input.loc[:5]
#print(test_input)

body = tag_data_with_langchain(test_input, llm, body_chat)

output=test_input
output['body']=body
output.to_csv("data/test/outsample_tag1.csv")


0
facing left
not body
1
head to the right
body
2
over his back
not body
3
leg out to the left
body
4
his arms are level with his head
body
5
his head comes past the right side of his body by a bit
body


/tmp/ipykernel_150043/924041610.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['body']=body
